# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Heqing_device1"

TIME_WINDOW = "30s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [3]:
sys_data_file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Heqing_device1/merged_data.npz')
assert sys_data_file_path.exists(), 'Data not found'
vocab_file = Path("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Heqing_device1/vocabulary.pkl")
assert vocab_file.exists(), f"Vocab file at {vocab_file} does not exist."

In [4]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [5]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [6]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
csv    530
zip     92
Name: filetype, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
FLS_data        91
KERN_data       91
RES_data        91
block_data      91
network_data    91
entropy_data    75
dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [7]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(127, 9)


### Get First TimeStamp

In [8]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

Error in file:  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/SYS_data/device1_thetick_1h_24.7G.zip
File is not a zip file


In [9]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-11-05 16:06:46 and 2023-11-13 17:38:29'

### Resample files

In [10]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [11]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

/tmp/ipykernel_102503/1828661645.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for source, group_df in df.groupby(["data_source"]):


Data type: DataSource.FileSystem
Shape: (239, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (120, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (120, 54)
Data type: DataSource.FileSystem
Shape: (122, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
same_df shape: (4203, 57)
Data type: DataSource.KernelEvents
Shape: (241, 56)
Data type: DataSource.

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [12]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

Reindexing only valid with uniquely valued Index objects


In [13]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['ext4:ext4_releasepage', 'jbd2:jbd2_checkpoint_stats', 'ext4:ext4_unlink_enter', 'writeback:sb_clear_inode_writeback', 'jbd2:jbd2_start_commit', 'block:block_bio_remap', 'ext4:ext4_writepages_result', 'seconds', 'ext4:ext4_da_update_reserve_space', 'block:block_bio_backmerge', 'block:block_dirty_buffer', 'block:block_getrq', 'writeback:writeback_dirty_inode', 'writeback:writeback_mark_inode_dirty', 'jbd2:jbd2_handle_start', 'ext4:ext4_free_inode', 'filemap:mm_filemap_delete_from_page_cache', 'writeback:writeback_dirty_inode_enqueue', 'writeback:global_dirty_state', 'ext4:ext4_es_lookup_extent_enter', 'jbd2:jbd2_handle_stats', 'writeback:writeback_dirty_page', 'block:block_unplug', 'ext4:ext4_journal_start', 'block:block_touch_buffer', 'filemap:mm_filemap_add_to_page_cache', 'label', 'device', 'data_source', 'sched:sched_process_exec', 'cpu-migrations', 'gpio:gpio_value', 'kmem:mm_page_free', 'timer:timer_start', 'kmem:kmem_cache_alloc', 'tcp:tcp_probe', 'kmem:kfree', 'kmem

In [14]:
new_dfs = []
for adf in df_mean_collection:
    print(adf.shape)
    adf = adf[~adf.index.duplicated(keep='first')]
    new_dfs.append(adf)

(4203, 54)
(4214, 56)
(4212, 89)
(4340, 3)
(4212, 2)


In [15]:
df_all = pd.concat(new_dfs, axis=1)
df_all.sample(2)

,ext4:ext4_releasepage_FLS_data,jbd2:jbd2_checkpoint_stats_FLS_data,ext4:ext4_discard_preallocations,ext4:ext4_unlink_enter_FLS_data,block:block_rq_insert,ext4:ext4_truncate_exit,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_commit_logging,jbd2:jbd2_start_commit_FLS_data,jbd2:jbd2_commit_locking,...,branch-load-misses,armv7_cortex_a15/l1d_tlb_refill/,timer:timer_init,branch-loads,cpu-cycles,write_ops,write_kbs,util,(OLD) VarianceLength,(OLD) AverageLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-10 20:00:46,1872.0,3.0,77.0,1.0,15.0,50.0,3.0,2.0,2.0,2.0,...,79529937.5,150872982.0,853.5,3.871848e+09,2.858031e+10,379.433333,4.200000,165.560000,291474.205958,1236.179063
2023-11-08 19:26:46,1.0,1.0,4.0,0.0,4.0,2.0,2.0,1.0,1.0,1.0,...,81511114.0,126130672.5,1072.5,3.584749e+09,2.554035e+10,3.466667,0.366667,6.346667,300939.435409,1230.463344


In [16]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [17]:
df_all.shape, len(SELECTED_FEATURES)

((21181, 205), 260)

In [18]:
df_all

,ext4:ext4_releasepage_FLS_data,jbd2:jbd2_checkpoint_stats_FLS_data,ext4:ext4_discard_preallocations,ext4:ext4_unlink_enter_FLS_data,block:block_rq_insert,ext4:ext4_truncate_exit,writeback:sb_clear_inode_writeback_FLS_data,jbd2:jbd2_commit_logging,jbd2:jbd2_start_commit_FLS_data,jbd2:jbd2_commit_locking,...,armv7_cortex_a15/l1d_tlb_refill/,timer:timer_init,branch-loads,cpu-cycles,write_ops,write_kbs,util,(OLD) VarianceLength,(OLD) AverageLength,label
2023-11-05 16:06:46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.5,0.5,...,9.514231e+07,330.0,2.834487e+09,1.943312e+10,3.466667,0.533333,11.333333,322350.751860,1197.459199,1_normal
2023-11-05 16:06:46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.5,0.5,...,9.514231e+07,330.0,2.834487e+09,1.943312e+10,3.466667,0.533333,11.333333,322350.751860,1197.459199,1_normal
2023-11-05 16:06:46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.5,0.5,...,9.514231e+07,330.0,2.834487e+09,1.943312e+10,3.466667,0.533333,11.333333,322350.751860,1197.459199,1_normal
2023-11-05 16:06:46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.5,0.5,...,9.514231e+07,330.0,2.834487e+09,1.943312e+10,3.466667,0.533333,11.333333,322350.751860,1197.459199,1_normal
2023-11-05 16:06:46,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.5,0.5,...,9.514231e+07,330.0,2.834487e+09,1.943312e+10,3.466667,0.533333,11.333333,322350.751860,1197.459199,1_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 17:37:46,1.0,0.0,2.0,0.0,4.5,1.0,1.0,0.0,0.0,0.0,...,1.069918e+08,1100.0,3.023709e+09,2.277917e+10,2.933333,0.400000,9.333333,383705.902655,1123.992119,9_xmrig
2023-11-13 17:37:46,1.0,0.0,2.0,0.0,4.5,1.0,1.0,0.0,0.0,0.0,...,1.069918e+08,1100.0,3.023709e+09,2.277917e+10,2.933333,0.400000,9.333333,383705.902655,1123.992119,9_xmrig
2023-11-13 17:37:46,1.0,0.0,2.0,0.0,4.5,1.0,1.0,0.0,0.0,0.0,...,1.069918e+08,1100.0,3.023709e+09,2.277917e+10,2.933333,0.400000,9.333333,383705.902655,1123.992119,9_xmrig
2023-11-13 17:37:46,1.0,0.0,2.0,0.0,4.5,1.0,1.0,0.0,0.0,0.0,...,1.069918e+08,1100.0,3.023709e+09,2.277917e+10,2.933333,0.400000,9.333333,383705.902655,1123.992119,9_xmrig


# Add Sys Call Data

In [19]:
def read_in_preprocessed_Xi_sensor_2():
    with np.load(sys_data_file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer(vocab_file=vocab_file)
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Xi_sensor_2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(11171, 147)
Loaded set: ('accept', 'accept4', 'access', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_create1', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fchdir', 'fchmod', 'fchown32', 'fcntl64', 'fgetxattr', 'flistxattr', 'fsetxattr', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getpriority', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'gettimeofday', 'getuid32', 'getxattr', 'inotify_add_watch', 'ioctl', 'ioprio_get', 'kcmp', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lseek', 'lstat64', 'madvise', 'mkdir', 'mmap2', 'mprotect', 'mremap', 'munmap', 'newselect', 'open', 'openat', 'pause', 'perf_event_open', '

In [20]:
sys_df.head(2)

,accept,accept4,access,adjtimex,bind,brk,capget,capset,chdir,chmod,...,uname,unlink,unlinkat,utimensat,utimes,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-05 16:06:46,0.0,0.000000,0.000655,0.0,0.000032,0.001087,0.000017,0.000007,0.0,0.0,...,0.000179,0.0,0.0,0.0,0.0,0.000321,0.0,0.027546,0.0,1_normal
2023-11-05 16:06:57,0.0,0.000014,0.000717,0.0,0.000004,0.000593,0.000000,0.000000,0.0,0.0,...,0.000176,0.0,0.0,0.0,0.0,0.000305,0.0,0.030063,0.0,1_normal


In [21]:
if sys_df["label"].unique().shape[0] != 9:
    raise Exception(f'Not all labels!!!: {sys_df["label"].unique()}')
sys_df["label"].unique()

array(['1_normal', '3_thetick', '2_ransomware', '4_bashlite',
       '5_httpbackdoor', '6_beurk', '7_backdoor', '8_bdvl', '9_xmrig'],
      dtype=object)

In [22]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [23]:
sys_df_mean.head(2)

,chown32,dup2,fstatat64,setsid,getsockopt,rt_sigaction,sysinfo,faccessat,getcwd,prlimit64,...,inotify_add_watch,pipe,poll,lstat64,rename,geteuid32,getuid32,clock_gettime,setresgid32,set_tid_address
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-05 16:06:46,0.0,0.000192,0.000080,0.0,0.000521,0.001843,0.000000,0.000000,0.000004,0.0,...,0.004695,0.000131,0.000372,0.048299,0.0,0.000405,0.000374,0.019952,0.0,0.000036
2023-11-05 16:07:16,0.0,0.000203,0.000065,0.0,0.000823,0.002466,0.000003,0.000003,0.000018,0.0,...,0.000004,0.000141,0.000580,0.000364,0.0,0.000458,0.000408,0.030682,0.0,0.000049


In [24]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (40183, 306)


In [25]:
all_df1.head(4)

,ext4:ext4_begin_ordered_truncate,jbd2:jbd2_start_commit_FLS_data,ext4:ext4_da_write_end,ext4:ext4_truncate_exit,jbd2:jbd2_commit_locking,ext4:ext4_invalidatepage,writeback:writeback_dirty_inode_enqueue_FLS_data,jbd2:jbd2_commit_logging,ext4:ext4_ext_remove_space,writeback:global_dirty_state_FLS_data,...,mmap2,timerfd_settime,setxattr,shutdown,epoll_create1,rmdir,mkdir,getuid32,llseek,ioctl
2023-11-05 16:06:46,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0,1.0,...,0.003921,0.000119,0.0,0.000003,0.0,0.0,0.0,0.000374,0.033152,0.05532
2023-11-05 16:06:46,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0,1.0,...,0.003921,0.000119,0.0,0.000003,0.0,0.0,0.0,0.000374,0.033152,0.05532
2023-11-05 16:06:46,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0,1.0,...,0.003921,0.000119,0.0,0.000003,0.0,0.0,0.0,0.000374,0.033152,0.05532
2023-11-05 16:06:46,0.0,0.5,0.0,0.0,0.5,0.0,0.5,0.5,0.0,1.0,...,0.003921,0.000119,0.0,0.000003,0.0,0.0,0.0,0.000374,0.033152,0.05532


In [26]:
spath = repo_base_path / "training" / "data" / "all_30s" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))